In [ ]:
import mlflow
import pandas as pd

uri = ... # Set your MLflow tracking URI here
mlflow.set_tracking_uri(uri)


In [ ]:

def generate_recommendations_with_comparision(
    experiment_ids,
    aggregation_function="common_features",
    main_note="sizes_acts",
    note_to_compare="sizes_L2_without_acts",
    group_type="sim"
):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            output_format="list"
        )

        for run in runs:
            note = run.data.params.get("note")
            fusion = run.data.params.get("SAE_fusion_strategy")
            group = run.data.params.get("group_type")

            if fusion != aggregation_function or group != group_type:
                continue

            dataset = run.data.params.get("dataset", f"Exp-{exp_id}")
            dim = int(run.data.params.get("embedding_dim", 0))
            topk = int(run.data.params.get("top_k", 0))

            row_key = (dim, topk)
            model_type = "main" if note == main_note else "compare" if note == note_to_compare else None
            if not model_type:
                continue

            metrics = {
                (dataset, "G/mean", model_type): run.data.metrics.get("CommonItemsNDCG20/mean"),
                (dataset, "U/mean", model_type): run.data.metrics.get("NDCG20/mean"),
                (dataset, "U/min", model_type): run.data.metrics.get("NDCG20/min"),
                (dataset, "Pop", model_type): run.data.metrics.get("Popularity/mean"),
            }

            all_rows.append((row_key, metrics))

    # Combine metrics into a dictionary
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    
    df.index.names = ["Dimensions", "TopK"]

    # Calculate % differences
    result_cols = []
    for dataset, metric in sorted(set((k[0], k[1]) for k in df.columns)):
        main_col = (dataset, metric, "main")
        compare_col = (dataset, metric, "compare")
        percent_col = (dataset, metric, "% change")

        if main_col in df.columns and compare_col in df.columns:
            df[percent_col] = ((df[main_col] - df[compare_col]) / df[compare_col].abs()) * 100
            result_cols.extend([percent_col])
        elif main_col in df.columns:
            result_cols.append(main_col)

    # Keep only main and percent change columns, sort them by metric
    df = df[result_cols]
    df = df.round(2)
    df = df.sort_index(axis=1, level=0).sort_values(by=["Dimensions", "TopK"])
    
        # Custom column sorting
    dataset_order = ["MovieLens", "LastFM1k"]
    metric_order = ["G/mean", "U/min", "U/mean", "Pop"]

    cols = df.columns
    cols = sorted(
        cols,
        key=lambda x: (
            dataset_order.index(x[0]) if x[0] in dataset_order else len(dataset_order),
            metric_order.index(x[1]) if x[1] in metric_order else len(metric_order)
        )
    )
    df = df[cols]
    

    return df.reset_index()

In [2]:
import mlflow
import pandas as pd

def generate_sae_table_with_comparison(experiment_ids, main_note, note_to_compare):
    all_rows = []

    for exp_id in experiment_ids:
        runs = mlflow.search_runs(
            experiment_ids=[exp_id],
            output_format="list"
        )

        for run in runs:
            note = run.data.params.get("note")
            if note not in [main_note, note_to_compare]:
                continue

            dataset = run.data.params.get("dataset", f"Exp-{exp_id}")
            dim = int(run.data.params.get("embedding_dim", 0))
            base_factors = float(run.data.params.get("base_factors", 0.0))
            expansion_ratio = dim / base_factors if base_factors else None
            topk = int(run.data.params.get("top_k", 0))

            row_key = (dim, round(expansion_ratio, 3), topk)
            model_type = "main" if note == main_note else "compare"

            metrics = {
                (dataset, model_type, "CS"): run.data.metrics.get("CosineSim/test"),
                (dataset, model_type, "Deg"): run.data.metrics.get("NDCG20_Degradation/test"),
                (dataset, model_type, "Deads"): run.data.metrics.get("DeadNeurons/test")
            }

            all_rows.append((row_key, metrics))

    # Build DataFrame
    records = {}
    for key, metrics in all_rows:
        if key not in records:
            records[key] = {}
        records[key].update(metrics)

    df = pd.DataFrame.from_dict(records, orient="index")
    df.index.names = ["Dimensions", "Expansion Ratio", "TopK"]

    # Calculate % differences and arrange columns
    result_cols = []
    for (dataset, metric) in sorted(set((k[0], k[2]) for k in df.columns)):
        main_col = (dataset, "main", metric)
        compare_col = (dataset, "compare", metric)
        percent_col = (dataset, "% change", metric)

        if main_col in df.columns and compare_col in df.columns:
            df[percent_col] = ((df[main_col] - df[compare_col]) / df[compare_col].abs()) * 100
            result_cols.extend([percent_col])
        elif main_col in df.columns:
            result_cols.append(main_col)

    # Finalize view: sort columns, round values
    df = df[result_cols]
    df = df.round(3)
    df = df.sort_index(axis=1, level=[0, 2]).sort_values(
        by=["Dimensions", "Expansion Ratio", "TopK"],
        ascending=[True, True, True]
    )

    return df.reset_index()

# 5. Contrastive Loss

Ackkoliv podle predchozich pruzkumu ukazuji autoencodery na specificke rysy, trenink probiha bez jakekoliv o to to zajistit. Vyzkum ... ukazuje metodu ...


Pojdme si ukazat jak se to promitne na vysledcich 

## Table of reconstrictions metrics

compare = L2, main = With contrastive loss in ration 0.5/0.3

- CS - Cosine Similarity of original and reconstructed embeddings
- Deads - Percentage of dead neurons in the sparse embedding
- Deg - Degradation of NDCG between ELSA model and ELSA + Autoencoder

In [3]:
sae_experiments = ['657713966175362303', '852893065079987597']

table = generate_sae_table_with_comparison(
    sae_experiments,
    main_note="sizes_constrastive_3",
    note_to_compare="sizes_L2",
)
table

Dimensions Expansion Ratio TopK LastFM1k                  MovieLens  \
                                  % change                   % change   
                                        CS   Deads      Deg        CS   
0       1024             4.0   32    0.094  40.435  -68.769    -0.422   
1       1024             4.0   64    0.035   9.412   26.511    -0.789   
2       1024             4.0  128   -0.021  -7.692   49.849    -1.207   
3       2048             8.0   32    0.114   7.064 -755.490    -0.012   
4       2048             8.0   64    0.071  26.265   26.102    -0.597   
5       2048             8.0  128   -0.057  27.778   24.180    -1.208   
6       4096            16.0   32    0.081   2.825 -154.842    -0.001   
7       4096            16.0   64   -0.034   5.885  872.898    -0.290   
8       4096            16.0  128   -0.171  13.457   81.953    -0.529   

                     
                     
     Deads      Deg  
0      NaN   -5.818  
1      NaN  -25.271  
2      NaN  -78.313  
3      NaN   10.260  
4      NaN  -21.031  
5      NaN -145.216  
6  135.714   15.073  
7      NaN    0.810  
8      NaN  -33.132

In [4]:
table.mean()

Dimensions                          2389.333333
Expansion Ratio                        9.333333
TopK                                  74.666667
LastFM1k         % change  CS          0.012444
                           Deads      13.936556
                           Deg        11.376889
MovieLens        % change  CS         -0.561667
                           Deads     135.714000
                           Deg       -31.404222
dtype: float64

- zvysuje se pocet dead neurons
- jinak hodnoty jsou si hodne podobny

## Table of reconstrictions metrics

compare = L2, main = With contrastive loss in ration **0.5/0.5**

- CS - Cosine Similarity of original and reconstructed embeddings
- Deads - Percentage of dead neurons in the sparse embedding
- Deg - Degradation of NDCG between ELSA model and ELSA + Autoencoder

In [5]:

sae_experiments = ['657713966175362303', '852893065079987597']

table = generate_sae_table_with_comparison(
    sae_experiments,
    main_note="sizes_constrastive_5",
    note_to_compare="sizes_L2",
)

In [6]:
table.mean()


Dimensions                          2389.333333
Expansion Ratio                        9.333333
TopK                                  74.666667
LastFM1k         % change  CS          0.122111
                           Deads      22.334111
                           Deg       135.271556
MovieLens        % change  CS         -1.107222
                           Deads            inf
                           Deg       -49.594667
dtype: float64

Temer totozne vysledky
- pro CS skoro stejne
- pro deg pro last lepsi, pro echo nest trochu horsi

## SAE group recommendation performance for **common features** aggregation function and **similar** groups

Comparing the basic variant (compare) with the contrastive loss coefficient **0.3**

In [7]:
experiments = ['523100174176986081', '333391697323445885']
table = generate_recommendations_with_comparision(
    experiments,
    aggregation_function="common_features",
    main_note="sizes_constrastive_3_without_acts",
    note_to_compare="sizes_L2_without_acts",
    group_type="sim",
)

## SAE group recommendation performance for **common features** aggregation function and **similar** groups

Comparing the basic variant (compare) with the contrastive loss coefficient **0.5**

In [12]:
experiments = ['523100174176986081', '333391697323445885']
total_table = pd.DataFrame()

def format_latex(df, highlight_max_cols=None, highlight_min_cols=None, round_digits=3):
    formatted_df = df.copy()
    highlight_max_cols = highlight_max_cols or []
    highlight_min_cols = highlight_min_cols or []

    for col in df.columns:
        col_values = df[col]

        if col in highlight_max_cols:
            top_two = col_values.nlargest(2).values

            def format_cell(val):
                if val == top_two[0]:
                    return f"\\textbf{{{val:.{round_digits}f}}}"
                elif val == top_two[1]:
                    return f"\\underline{{{val:.{round_digits}f}}}"
                else:
                    return f"{val:.{round_digits}f}"

        elif col in highlight_min_cols:
            bottom_two = col_values.nsmallest(2).values

            def format_cell(val):
                if val == bottom_two[0]:
                    return f"\\textbf{{{val:.{round_digits}f}}}"
                elif val == bottom_two[1]:
                    return f"\\underline{{{val:.{round_digits}f}}}"
                else:
                    return f"{val:.{round_digits}f}"
        else:
            def format_cell(val):
                return f"{val}"

        formatted_df[col] = col_values.apply(format_cell)

    return formatted_df

highlight_max_cols = [('MovieLens', 'G/mean', r"% change"), ('MovieLens', 'U/min', r"% change"), ('MovieLens', 'U/mean', r"% change"), ('LastFM1k', 'G/mean', r"% change"), ('LastFM1k', 'U/min', r"% change"), ('LastFM1k', 'U/mean', r"% change")]
highlight_min_cols = [('MovieLens', 'Pop', r"% change"), ('LastFM1k', 'Pop', r"% change")]

group_types = ['sim', 'random', 'outlier']
variants = [
    ('common_features', 'sizes_constrastive_3_without_acts', 'sizes_L2_without_acts'),
    ('common_features', 'sizes_constrastive_5_without_acts', 'sizes_L2_without_acts'),
    ('average', 'sizes_constrastive_3_with_acts', 'sizes_L2_with_acts'),
    ('average', 'sizes_constrastive_5_with_acts', 'sizes_L2_with_acts'),
    ]

for group_type in group_types:
    for aggregation_function, main_note, note_to_compare in variants:
        table = generate_recommendations_with_comparision(
            experiments,
            aggregation_function=aggregation_function,
            main_note=main_note,
            note_to_compare=note_to_compare,
            group_type=group_type
        )

        table['Group Type'] = "Similar" if group_type == 'sim' else "Random" if group_type == 'random' else "Outlier"
        table['alpha'] = 1.0 if 'constrastive_5' in main_note else 0.5
        table['Aggregation'] = "ComF" if aggregation_function == 'common_features' else "AVG"
        table = table.groupby(['Group Type', 'alpha', 'Aggregation']).mean().reset_index()

        total_table = pd.concat([total_table, table], ignore_index=True)
        
        print(f"Processed {aggregation_function} with {group_type} grouping.")
        

total_table

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_50320/982618436.py:65: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table = table.groupby(['Group Type', 'alpha', 'Aggregation']).mean().reset_index()


Processed common_features with sim grouping.


/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_50320/982618436.py:65: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table = table.groupby(['Group Type', 'alpha', 'Aggregation']).mean().reset_index()


Processed common_features with sim grouping.


/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_50320/982618436.py:65: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table = table.groupby(['Group Type', 'alpha', 'Aggregation']).mean().reset_index()


...

Processed common_features with outlier grouping.


/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_50320/982618436.py:65: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table = table.groupby(['Group Type', 'alpha', 'Aggregation']).mean().reset_index()


Processed average with outlier grouping.
Processed average with outlier grouping.


/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_50320/982618436.py:65: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table = table.groupby(['Group Type', 'alpha', 'Aggregation']).mean().reset_index()


Group Type alpha Aggregation   Dimensions       TopK  MovieLens             \
                                                            G/mean      U/min   
                                                          % change   % change   
0     Similar   0.5        ComF  2389.333333  74.666667 -17.620000 -15.158889   
1     Similar   1.0        ComF  2389.333333  74.666667 -25.027778 -21.932222   
2     Similar   0.5         AVG  2389.333333  74.666667   0.036667   0.068889   
3     Similar   1.0         AVG  2389.333333  74.666667   0.001111  -0.098889   
4      Random   0.5        ComF  2389.333333  74.666667 -28.148889 -25.655556   
5      Random   1.0        ComF  2389.333333  74.666667 -40.084444 -36.683333   
6      Random   0.5         AVG  2389.333333  74.666667  -0.591111  -0.391111   
7      Random   1.0         AVG  2389.333333  74.666667  -0.936667  -0.792222   
8     Outlier   0.5        ComF  2389.333333  74.666667 -39.608889 -36.775556   
9     Outlier   1.0        ComF  2389.333333  74.666667 -58.225556 -53.644444   
10    Outlier   0.5         AVG  2389.333333  74.666667  -3.722222  -3.158889   
11    Outlier   1.0         AVG  2389.333333  74.666667  -5.115556  -4.298889   

                          LastFM1k                                
       U/mean        Pop    G/mean     U/min    U/mean       Pop  
     % change   % change  % change  % change  % change  % change  
0  -11.956667 -12.455556 -0.173333 -0.255556 -0.105556  1.264444  
1  -17.788889 -13.278889 -0.283333 -0.435556 -0.152222  1.741111  
2    0.063333  -2.405556  0.002222  0.015556  0.013333 -0.220000  
3    0.115556  -3.115556  0.150000  0.170000  0.078889 -0.131111  
4  -20.605556 -18.122222  0.290000  0.192222  0.112222  1.287778  
5  -30.775556 -23.695556  0.211111 -0.026667  0.066667  2.053333  
6   -0.193333  -2.708889 -0.223333 -0.094444 -0.101111 -0.221111  
7   -0.326667  -3.650000  0.006667  0.125556 -0.001111 -0.152222  
8  -28.782222 -18.635556  1.826667  1.031111  0.562222  2.440000  
9  -42.796667 -28.616667  2.055556  0.998889  0.624444  3.660000  
10  -1.365556  -4.840000 -0.165556 -0.238889 -0.093333 -0.472222  
11  -1.836667  -6.444444  0.163333  0.076667  0.052222 -0.364444

In [13]:
total_table = total_table.drop(columns=['Dimensions', 'TopK']).set_index(['Aggregation', 'Group Type', 'alpha']).sort_index()

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_50320/1005546430.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  total_table = total_table.drop(columns=['Dimensions', 'TopK']).set_index(['Aggregation', 'Group Type', 'alpha']).sort_index()


In [14]:
total_table

MovieLens                                   \
                                 G/mean      U/min     U/mean        Pop   
                               % change   % change   % change   % change   
Aggregation Group Type alpha                                               
AVG         Outlier    0.5    -3.722222  -3.158889  -1.365556  -4.840000   
                       1.0    -5.115556  -4.298889  -1.836667  -6.444444   
            Random     0.5    -0.591111  -0.391111  -0.193333  -2.708889   
                       1.0    -0.936667  -0.792222  -0.326667  -3.650000   
            Similar    0.5     0.036667   0.068889   0.063333  -2.405556   
                       1.0     0.001111  -0.098889   0.115556  -3.115556   
ComF        Outlier    0.5   -39.608889 -36.775556 -28.782222 -18.635556   
                       1.0   -58.225556 -53.644444 -42.796667 -28.616667   
            Random     0.5   -28.148889 -25.655556 -20.605556 -18.122222   
                       1.0   -40.084444 -36.683333 -30.775556 -23.695556   
            Similar    0.5   -17.620000 -15.158889 -11.956667 -12.455556   
                       1.0   -25.027778 -21.932222 -17.788889 -13.278889   

                              LastFM1k                                
                                G/mean     U/min    U/mean       Pop  
                              % change  % change  % change  % change  
Aggregation Group Type alpha                                          
AVG         Outlier    0.5   -0.165556 -0.238889 -0.093333 -0.472222  
                       1.0    0.163333  0.076667  0.052222 -0.364444  
            Random     0.5   -0.223333 -0.094444 -0.101111 -0.221111  
                       1.0    0.006667  0.125556 -0.001111 -0.152222  
            Similar    0.5    0.002222  0.015556  0.013333 -0.220000  
                       1.0    0.150000  0.170000  0.078889 -0.131111  
ComF        Outlier    0.5    1.826667  1.031111  0.562222  2.440000  
                       1.0    2.055556  0.998889  0.624444  3.660000  
            Random     0.5    0.290000  0.192222  0.112222  1.287778  
                       1.0    0.211111 -0.026667  0.066667  2.053333  
            Similar    0.5   -0.173333 -0.255556 -0.105556  1.264444  
                       1.0   -0.283333 -0.435556 -0.152222  1.741111

In [20]:
format_latex(
    total_table,
    highlight_max_cols=highlight_max_cols,
    highlight_min_cols=highlight_min_cols,
    round_digits=2
).to_latex(
    "sae_table.tex",
    index=True,
    float_format="%.2f",
    bold_rows=False,
    column_format="lll|rrrr|rrrr",
    escape=False,
    caption = (
        "Table showing the percentage change in recommendation metrics for different weight of contrastive loss (alpha), for different aggregation strategies (ComF - Common Features, AVG - Average) and group types (Similar, Random, Outlier)."
        "'G/mean' shows the percentage change in mean NDCG@20 using ground-truth recommendations seen by all group members. "
        "'U/min' shows the change in the mean of the minimum NDCG@20 across group members. "
        "'U/mean' shows the change in the mean of the average NDCG@20 across group members. "
        "'Pop' shows the change in the mean popularity of recommended items."
    ),
    label="tab:contrastive:recommendation"
)

## SAE group recommendation performance for **average** aggregation function and **similar** groups

Comparing the basic variant (compare) with the contrastive loss coefficient **0.3**

In [49]:
experiments = ['523100174176986081', '333391697323445885']
table = generate_recommendations_with_comparision(
    experiments,
    aggregation_function="average",
    main_note="sizes_constrastive_5_without_acts",
    note_to_compare="sizes_L2_without_acts",
    group_type="outlier",
)
table

Dimensions TopK MovieLens                            LastFM1k           \
                     G/mean    U/min   U/mean      Pop   G/mean    U/min   
                   % change % change % change % change % change % change   
0       1024   32      4.82     2.18     1.70     6.29     1.11    -0.03   
1       1024   64      1.65     1.16     2.05     1.64     3.44     1.91   
2       1024  128     -1.38    -1.27     0.23    -2.62     6.19     4.42   
3       2048   32    108.98    79.48    39.98    87.42    -0.18    -0.47   
4       2048   64     35.84    27.94    17.33    37.68    -0.07    -0.84   
5       2048  128     16.11    11.85     8.64    17.57     3.78     2.18   
6       4096   32     94.67    69.65    35.98    68.52    -0.62    -1.38   
7       4096   64     79.80    56.26    31.63    68.76    -0.52    -1.76   
8       4096  128     28.23    20.38    13.14    28.35     1.38    -0.26   

                     
    U/mean      Pop  
  % change % change  
0     0.28     2.08  
1     0.76     4.15  
2     1.61     5.53  
3    -0.08     0.67  
4     0.07     2.20  
5     1.16     5.93  
6    -0.06    -0.10  
7    -0.20     2.10  
8     0.44     4.52

In [ ]:
total_table = table.mean().round(2)
total_table['Group Type'] = 'Similar'
total_table['alpha'] = 0.5
total_table['Aggregation'] = 'Average'

total_table

Dimensions                       2389.33
TopK                               74.67
MovieLens    G/mean  % change     -17.62
             U/min   % change     -15.16
             U/mean  % change     -11.96
             Pop     % change     -12.46
LastFM1k     G/mean  % change      -0.17
             U/min   % change      -0.26
             U/mean  % change      -0.11
             Pop     % change       1.26
Group Type                       Similar
alpha                                0.5
Aggregation                      Average
dtype: object

In [ ]:
table.groupby("Dataset").mean().round(2)

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_61516/3289877605.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby("Dataset").mean().round(2)


Dimensions   TopK   G/mean                    Pop                \
                            % change compare  main % change compare  main   
Dataset                                                                     
LastFM1k     2389.33  74.67     0.00    0.59  0.59    -0.22    0.57  0.57   
MovieLens    2389.33  74.67     0.04    0.65  0.65    -2.41    0.50  0.49   

            U/mean                  U/min                
          % change compare  main % change compare  main  
Dataset                                                  
LastFM1k      0.01     0.8  0.81     0.02    0.64  0.64  
MovieLens     0.06     0.7  0.70     0.07    0.56  0.56

## SAE group recommendation performance for **average** aggregation function and **similar** groups

Comparing the basic variant (compare) with the contrastive loss coefficient **0.5**

In [ ]:
experiments = ['523100174176986081', '333391697323445885']
table = generate_recommendations_with_comparision(
    experiments,
    aggregation_function="average",
    main_note="sizes_constrastive_5_with_acts",
    note_to_compare="sizes_L2_with_acts",
    group_type="sim",
)
table

Dataset Dimensions TopK   G/mean                    Pop                \
                              % change compare  main % change compare  main   
0    LastFM1k       1024   32     0.88    0.58  0.59    -0.53    0.57  0.57   
1    LastFM1k       1024   64     0.54    0.58  0.59    -0.22    0.57  0.57   
2    LastFM1k       1024  128     0.90    0.58  0.59     0.59    0.56  0.57   
3    LastFM1k       2048   32     0.45    0.59  0.59    -0.37    0.57  0.57   
4    LastFM1k       2048   64    -0.01    0.59  0.59    -0.33    0.57  0.57   
5    LastFM1k       2048  128     0.23    0.59  0.59     0.21    0.57  0.57   
6    LastFM1k       4096   32    -1.07    0.60  0.59    -0.04    0.57  0.57   
7    LastFM1k       4096   64    -0.55    0.60  0.60    -0.18    0.57  0.57   
8    LastFM1k       4096  128    -0.02    0.59  0.59    -0.31    0.57  0.57   
9   MovieLens       1024   32     0.45    0.64  0.65    -1.68    0.50  0.49   
10  MovieLens       1024   64    -0.35    0.65  0.64    -3.84    0.50  0.48   
11  MovieLens       1024  128    -0.20    0.65  0.65    -2.72    0.50  0.49   
12  MovieLens       2048   32    -1.16    0.65  0.64    -6.26    0.50  0.47   
13  MovieLens       2048   64     0.07    0.65  0.65    -2.94    0.50  0.49   
14  MovieLens       2048  128    -0.24    0.65  0.65    -2.11    0.50  0.49   
15  MovieLens       4096   32     0.56    0.64  0.65    -3.75    0.51  0.49   
16  MovieLens       4096   64     0.60    0.64  0.65    -3.20    0.50  0.49   
17  MovieLens       4096  128     0.28    0.65  0.65    -1.54    0.50  0.49   

     U/mean                  U/min                
   % change compare  main % change compare  main  
0      0.23    0.80  0.80     0.96    0.63  0.63  
1      0.26    0.80  0.80     0.62    0.63  0.63  
2      0.27    0.80  0.80     0.67    0.63  0.64  
3      0.16    0.81  0.81     0.10    0.64  0.64  
4      0.11    0.80  0.81     0.02    0.64  0.64  
5      0.09    0.80  0.81     0.26    0.64  0.64  
6     -0.32    0.81  0.81    -1.05    0.64  0.64  
7     -0.09    0.81  0.81    -0.33    0.64  0.64  
8      0.00    0.81  0.81     0.28    0.64  0.64  
9      0.24    0.70  0.70     0.11    0.56  0.56  
10    -0.10    0.70  0.70    -0.28    0.56  0.56  
11    -0.02    0.70  0.70    -0.27    0.56  0.56  
12    -0.38    0.70  0.70    -1.00    0.56  0.55  
13     0.22    0.70  0.70    -0.26    0.56  0.56  
14     0.02    0.70  0.70    -0.04    0.56  0.56  
15     0.40    0.70  0.70     0.24    0.56  0.56  
16     0.33    0.70  0.70     0.29    0.56  0.56  
17     0.33    0.70  0.71     0.32    0.56  0.56

- horsi

In [ ]:
table.groupby("Dataset").mean().round(2)

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_61516/3289877605.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby("Dataset").mean().round(2)


Dimensions   TopK   G/mean                    Pop                \
                            % change compare  main % change compare  main   
Dataset                                                                     
LastFM1k     2389.33  74.67     0.15    0.59  0.59    -0.13    0.57  0.57   
MovieLens    2389.33  74.67     0.00    0.65  0.65    -3.12    0.50  0.49   

            U/mean                  U/min                
          % change compare  main % change compare  main  
Dataset                                                  
LastFM1k      0.08     0.8  0.81     0.17    0.64  0.64  
MovieLens     0.12     0.7  0.70    -0.10    0.56  0.56

## SAE group recommendation performance for **average** aggregation function and **random** groups

Comparing the basic variant (compare) with the contrastive loss coefficient **0.3**

In [ ]:
experiments = ['523100174176986081', '333391697323445885']
table = generate_recommendations_with_comparision(
    experiments,
    aggregation_function="average",
    main_note="sizes_constrastive_3_with_acts",
    note_to_compare="sizes_L2_with_acts",
    group_type="random",
)
table

Dataset Dimensions TopK   G/mean                    Pop                \
                              % change compare  main % change compare  main   
0    LastFM1k       1024   32     0.41    0.49  0.50    -0.32    0.64  0.64   
1    LastFM1k       1024   64     0.02    0.49  0.49    -0.42    0.63  0.63   
2    LastFM1k       1024  128     0.84    0.49  0.50     0.14    0.63  0.63   
3    LastFM1k       2048   32    -0.57    0.50  0.50    -0.18    0.64  0.64   
4    LastFM1k       2048   64     0.30    0.50  0.50    -0.28    0.64  0.64   
5    LastFM1k       2048  128    -1.28    0.50  0.49    -0.44    0.63  0.63   
6    LastFM1k       4096   32    -0.91    0.51  0.50     0.07    0.64  0.64   
7    LastFM1k       4096   64    -0.49    0.51  0.51    -0.25    0.64  0.64   
8    LastFM1k       4096  128    -0.33    0.50  0.50    -0.31    0.64  0.64   
9   MovieLens       1024   32    -0.87    0.63  0.62    -4.14    0.55  0.53   
10  MovieLens       1024   64    -1.26    0.63  0.63    -3.33    0.55  0.53   
11  MovieLens       1024  128    -0.72    0.63  0.63    -2.14    0.54  0.53   
12  MovieLens       2048   32    -1.11    0.63  0.62    -4.78    0.55  0.52   
13  MovieLens       2048   64    -0.99    0.63  0.62    -3.11    0.54  0.53   
14  MovieLens       2048  128    -0.32    0.63  0.63    -1.67    0.54  0.53   
15  MovieLens       4096   32    -0.54    0.63  0.63    -3.32    0.55  0.53   
16  MovieLens       4096   64     0.32    0.63  0.63    -0.69    0.54  0.54   
17  MovieLens       4096  128     0.17    0.63  0.63    -1.20    0.54  0.53   

     U/mean                  U/min                
   % change compare  main % change compare  main  
0      0.03    0.75  0.75     0.31    0.54  0.54  
1      0.01    0.75  0.75     0.11    0.54  0.54  
2      0.21    0.75  0.75     0.43    0.54  0.54  
3     -0.22    0.75  0.75    -0.31    0.55  0.54  
4      0.14    0.75  0.75     0.58    0.54  0.55  
5     -0.49    0.75  0.75    -0.83    0.55  0.54  
6     -0.30    0.76  0.75    -0.44    0.55  0.55  
7     -0.15    0.76  0.76    -0.42    0.55  0.55  
8     -0.14    0.75  0.75    -0.28    0.55  0.54  
9     -0.24    0.69  0.68    -0.67    0.54  0.53  
10    -0.46    0.69  0.69    -1.27    0.54  0.53  
11    -0.32    0.69  0.69    -0.51    0.54  0.54  
12    -0.45    0.69  0.68    -0.78    0.54  0.53  
13    -0.36    0.69  0.69    -0.59    0.54  0.54  
14    -0.07    0.69  0.69     0.04    0.54  0.54  
15    -0.22    0.69  0.69    -0.29    0.54  0.54  
16     0.12    0.69  0.69     0.29    0.54  0.54  
17     0.26    0.69  0.69     0.26    0.54  0.54

In [ ]:
table.groupby("Dataset").mean().round(2)

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_61516/3289877605.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby("Dataset").mean().round(2)


Dimensions   TopK   G/mean                    Pop                \
                            % change compare  main % change compare  main   
Dataset                                                                     
LastFM1k     2389.33  74.67    -0.22    0.50  0.50    -0.22    0.64  0.64   
MovieLens    2389.33  74.67    -0.59    0.63  0.63    -2.71    0.54  0.53   

            U/mean                  U/min                
          % change compare  main % change compare  main  
Dataset                                                  
LastFM1k     -0.10    0.75  0.75    -0.09    0.55  0.54  
MovieLens    -0.19    0.69  0.69    -0.39    0.54  0.54

## SAE group recommendation performance for **average** aggregation function and **div** groups

Comparing the basic variant (compare) with the contrastive loss coefficient **0.3**

In [ ]:
experiments = ['523100174176986081', '333391697323445885']
table = generate_recommendations_with_comparision(
    experiments,
    aggregation_function="average",
    main_note="sizes_constrastive_3_with_acts",
    note_to_compare="sizes_L2_with_acts",
    group_type="outlier",
)
table

Dataset Dimensions TopK   G/mean                    Pop                \
                              % change compare  main % change compare  main   
0    LastFM1k       1024   32     0.09    0.41  0.41    -0.97    0.59  0.58   
1    LastFM1k       1024   64     1.04    0.40  0.40    -0.79    0.58  0.57   
2    LastFM1k       1024  128     0.11    0.40  0.40    -0.05    0.57  0.57   
3    LastFM1k       2048   32    -0.11    0.42  0.42     0.16    0.59  0.59   
4    LastFM1k       2048   64     0.86    0.41  0.42    -0.36    0.58  0.58   
5    LastFM1k       2048  128    -1.42    0.41  0.41    -1.04    0.58  0.57   
6    LastFM1k       4096   32    -0.99    0.43  0.42    -0.18    0.59  0.59   
7    LastFM1k       4096   64    -0.81    0.43  0.42    -0.47    0.58  0.58   
8    LastFM1k       4096  128    -0.26    0.42  0.41    -0.55    0.58  0.58   
9   MovieLens       1024   32    -5.10    0.54  0.52    -7.29    0.48  0.45   
10  MovieLens       1024   64    -5.74    0.55  0.52    -6.17    0.48  0.45   
11  MovieLens       1024  128    -3.24    0.55  0.53    -4.59    0.47  0.45   
12  MovieLens       2048   32    -7.75    0.55  0.51    -9.07    0.48  0.44   
13  MovieLens       2048   64    -4.02    0.54  0.52    -5.61    0.47  0.45   
14  MovieLens       2048  128    -2.85    0.54  0.53    -2.94    0.47  0.46   
15  MovieLens       4096   32    -4.25    0.55  0.53    -6.20    0.48  0.45   
16  MovieLens       4096   64    -0.58    0.55  0.54    -0.44    0.48  0.47   
17  MovieLens       4096  128     0.03    0.54  0.54    -1.25    0.47  0.46   

     U/mean                  U/min                
   % change compare  main % change compare  main  
0     -0.10    0.71  0.71    -0.90    0.45  0.45  
1      0.14    0.71  0.71     0.44    0.44  0.44  
2      0.17    0.71  0.71     0.01    0.44  0.44  
3     -0.06    0.72  0.72    -0.05    0.45  0.45  
4      0.24    0.71  0.72     0.55    0.45  0.45  
5     -0.50    0.71  0.71    -0.87    0.45  0.44  
6     -0.29    0.72  0.72    -0.73    0.46  0.45  
7     -0.37    0.72  0.72    -0.64    0.46  0.45  
8     -0.07    0.72  0.71     0.04    0.45  0.45  
9     -1.94    0.66  0.65    -4.72    0.49  0.46  
10    -2.18    0.67  0.65    -4.28    0.49  0.47  
11    -1.35    0.67  0.66    -2.73    0.49  0.48  
12    -2.83    0.67  0.65    -6.12    0.49  0.46  
13    -1.45    0.66  0.65    -3.72    0.49  0.47  
14    -0.96    0.66  0.66    -2.80    0.49  0.47  
15    -1.50    0.67  0.66    -3.61    0.49  0.47  
16    -0.03    0.67  0.67     0.06    0.49  0.49  
17    -0.05    0.66  0.66    -0.51    0.48  0.48

In [ ]:
table.groupby("Dataset").mean().round(2)

/var/folders/11/x3r2lw756bs10f4yq3hp5cqh0000gn/T/ipykernel_61516/3289877605.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table.groupby("Dataset").mean().round(2)


Dimensions   TopK   G/mean                    Pop                \
                            % change compare  main % change compare  main   
Dataset                                                                     
LastFM1k     2389.33  74.67    -0.17    0.41  0.41    -0.47    0.58  0.58   
MovieLens    2389.33  74.67    -3.72    0.55  0.53    -4.84    0.48  0.45   

            U/mean                  U/min                
          % change compare  main % change compare  main  
Dataset                                                  
LastFM1k     -0.09    0.71  0.71    -0.24    0.45  0.45  
MovieLens    -1.37    0.67  0.66    -3.16    0.49  0.47